In [12]:
using Revise

In [13]:
using RigidBodyDynamics
using LCPSim
using Gurobi
using JuMP
using LearningMPC
using ExplicitQPs
using Plots; gr()
using ProgressMeter

In [14]:
mechanism = parse_urdf(Float64, "cartpole.urdf")
world = root_body(mechanism)
env = Environment{Float64}(Dict())

LCPSim.Environment{Float64}(Dict{RigidBodyDynamics.RigidBody{Float64},LCPSim.ContactEnvironment{Float64}}())

In [15]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "slider_to_cart"), [0])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = zeros(num_velocities(x0))
xstar = MechanismState(mechanism, q0, v0);

contacts = Point3D[]
Q = diagm([10, 10, 1, 1])
R = 0.1 * eye(num_velocities(x0))
K, S = LCPSim.ContactLQR.contact_lqr(x0, u0, Q, R, contacts)

([-10.0 -124.333 -15.5397 -61.6417; 0.0 0.0 0.0 0.0], [15.5397 61.6417 11.5741 31.1853; 61.6417 378.595 64.6041 189.485; 11.5741 64.6041 11.7022 32.752; 31.1853 189.485 32.752 95.7494])

In [16]:
function run_optimization(x0, env, Δt, N; x_nominal=x0, solver=GurobiSolver())
    qstar = copy(configuration(x_nominal))
    vstar = zeros(num_velocities(x_nominal))
    ustar = u0
    
    model = Model(solver=solver)
    @variable model q0[1:num_positions(x0)]
    JuMP.fix.(q0, configuration(x0))
    @variable model v0[1:num_velocities(x0)]
    JuMP.fix.(v0, velocity(x0))
    
    _, results_opt = LCPSim.optimize(MechanismState(x0.mechanism, q0, v0), env, Δt, N, model)
    
    objective = (
        sum(Δt * (r.state.state' * Q * r.state.state + 0.01 * r.input' * R * r.input) for r in results_opt)
       + (results_opt[end].state.state' * S * results_opt[end].state.state)
        )

    for r in results_opt
        for (joint, jrs) in r.joint_contacts
            for joint_result in jrs
                objective += joint_result.λ^2
            end
        end 
    end
    
    # Ensure objective is strictly PD
    nvars = length(model.colCat)
    vars = [Variable(model, i) for i in 1:nvars]
    objective += QuadExpr(vars, vars, [1e-6 for v in vars], AffExpr([], [], 0.0))
    
    @objective model Min objective
    solve(model)
    results = getvalue.(results_opt)
    
    ConditionalJuMP.warmstart!(model, true)
    solve(model)
    @assert sum(model.colCat .== :Bin) == 0 "Model should no longer have any binary variables"
    exsol = ExplicitQPs.explicit_solution(model, vcat(q0, v0))
    J = ExplicitQPs.jacobian(exsol, results_opt[1].input)
    
    return results, J
end

run_optimization (generic function with 1 method)

In [17]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)

Δt = 0.01
gap = 1e-3
timelimit = 60
horizon = 10

θs = linspace(-0.05, 0.05, 15)
inputs = Float64[]
derivatives = Float64[]
@showprogress for θ in θs
    set_configuration!(x0, q0)
    set_velocity!(x0, v0)
    set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [θ])
    r_control, J = run_optimization(x0, env, Δt, horizon; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    push!(inputs, r_control[1].input[1])
    push!(derivatives, J[1, 2])
end


Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In [18]:
plt = plot(θs, inputs, line=nothing, marker=:circle, legend=nothing)
Δθ = (θs[2] - θs[1]) / 2
for (i, d) in enumerate(derivatives)
    θ = θs[i]
    input = inputs[i]
    plot!(plt, [θ - Δθ, θ + Δθ], [input - Δθ * d, input + Δθ * d])
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.04 
 
 
 -0.02 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20